# Sample for a Fisher Market with UMP with homogeneous affine constraints

- the current implementation is for concept proof only,
  it is far from efficient and scalable to large-scale problems

In [1]:
using Pkg
Pkg.activate("../")

  Activating project at `~/workspace/ExchangeMarket.jl/scripts`


In [2]:
using Revise
using SparseArrays, LinearAlgebra
using JuMP, MosekTools
import MathOptInterface as MOI
using Plots, LaTeXStrings, Printf, Random, GraphRecipes
using CSV, DataFrames, Graphs, Distributions

using ExchangeMarket

include("../tools.jl")
include("../plots.jl")
include("setup.jl")
switch_to_pdf(;)


:pdf

## Random instance

In [39]:
function generate_random_graph(nᵥ)
    # or create a small random graph
    E = []
    for i in 1:nᵥ
        for j in 1:nᵥ
            if i != j
                push!(E, (i, j))
            end
        end
    end
    nₑ = length(E)
    adj = sparse(first.(E), last.(E), ones(length(E)), nᵥ, nᵥ)
    # assign capacities and random revenues
    # the first element is for the output flow
    c = rand(Float64, nₑ + 1)
    r = rand(Uniform(1.0, 10.0), nₑ + 1)
    return adj, E, c, r, nᵥ, nₑ
end

adj, E, c, r, nᵥ, nₑ = generate_random_graph(15)


(sparse([2, 3, 4, 5, 6, 7, 8, 9, 10, 11  …  5, 6, 7, 8, 9, 10, 11, 12, 13, 14], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 15, 15), Any[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11)  …  (15, 5), (15, 6), (15, 7), (15, 8), (15, 9), (15, 10), (15, 11), (15, 12), (15, 13), (15, 14)], [0.6976256760456924, 0.5820079055668151, 0.9266416419470301, 0.16978311591900264, 0.11632148548362331, 0.6245787132525471, 0.5610200275764593, 0.48553433612344177, 0.398959587411735, 0.5958635553211585  …  0.908402551956242, 0.18011928817263578, 0.24307699400701388, 0.1777279398779763, 0.14461352871648214, 0.8262400208152666, 0.8782245422155183, 0.9737464834971311, 0.4113647049697803, 0.9815395722155631], [2.5495817537452328, 9.420843361226263, 5.413725912951675, 8.290346996390394, 2.141102167900297, 3.0881437550552375, 4.2494783437509, 1.4

In [40]:
m = 150
n = length(E)
stpairs = [
    zip(fill(1, m), rand(1:nᵥ, m))...
]
vec_const_x = []
for (s, t) in stpairs
    md, A, _... = ExchangeMarket.__query_Abc_standard(s, t, adj, E, nᵥ)
    mₐ, nₐ = size(A)
    push!(vec_const_x, LinearConstr(mₐ, nₐ, A, zeros(mₐ)))
end
ρ = 0.4
constr_p = LinearConstr(1, n, ones(1, n), [1.0])

f0 = FisherMarket(
    m, n; ρ=ρ,
    constr_x=vec_const_x, constr_p=constr_p,
    bool_unit=true, bool_unit_wealth=true,
    scale=10.0, sparsity=0.95
)

FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.0004107952117919922 seconds
FisherMarket initialized in 0.0004718303680419922 seconds


FisherMarket{Float64}(150, 210, [6.365987373e-314 2.6133912847e-314 … -0.002078294754056742 7.511857027e-314; 2.6133808145e-314 2.121995791e-314 … -0.0012998562306165695 0.0; … ; 2.6133816326e-314 2.613390078e-314 … 0.0 2.613505307e-314; 8.487983166e-314 2.121995791e-314 … 0.0 8.4879831644e-314], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.002019807643080559, 0.002400947230713352, 0.006268992962703984, 0.0005909799984515632, 0.00975635525834286, 0.01031344376522511, 0.011122301330159096, 4.4865351152652696e-5, 0.01000439591241609, 0.008402420778366617  …  0.006512125590101107, 0.0007405701349098392, 0.006783518858554462, 0.01219186835058268, 0.00015149077289232488, 0.0072295007956077835, 0.0029173507080273837, 0.007299964475887644, 0.004891303957205201, 0.012296905363855345], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [-0.0005523502457840609 -0.00052173495213110

In [41]:
# define optimization arguments
method = HessianBar
kwargs = Dict(
    :tol => 1e-10,
    :maxiter => 20,
    :optimizer => AFCESConic,
    # :option_mu => :pred_corr,
    :option_mu => :nothing,
    :option_step => :affinesc,
    # :option_step => :logbar,
    :linsys => :direct_affine, # only support direct for now
)

Dict{Symbol, Any} with 6 entries:
  :maxiter     => 20
  :tol         => 1.0e-10
  :optimizer   => ResponseOptimizer(__af_conic_log_response_ces, :linconicaffin…
  :option_mu   => :nothing
  :linsys      => :direct_affine
  :option_step => :affinesc

In [42]:
f1 = copy(f0)
p₀ = ones(n) * sum(f1.w) ./ (n)
if typeof(f1.c) <: SparseMatrixCSC
    x₀ = similar(f1.c)
    x₀.nzval .= 1.0
else
    x₀ = ones(n, m) ./ m
end
f1.x .= x₀
f1.p .= p₀
alg = method(
    n, m, p₀;
    linconstr=constr_p,
    μ=1e-5,
    kwargs...
)
traj = opt!(
    alg, f1;
    keep_traj=true,
    maxiter=30,
    maxtime=500.0,
    loginterval=1,
    tol=1e-12
)
validate(f1, alg)

FisherMarket initialization started...
FisherMarket cost matrix initialized in 7.700920104980469e-5 seconds
FisherMarket initialized in 0.0001239776611328125 seconds
--------------------------------------------------------------------------------------------
                   ExchangeMarket.jl: A Julia Package for Exchange Market                   
                                    © Chuwen Zhang (2024)                                    
--------------------------------------------------------------------------------------------
 subproblem solver alias       := AFCESConicResponse
 subproblem solver style       := linconicaffine
 lin-system solver alias       := direct_affine
 option for gradient           := dual
 option for step               := affinesc
 option for μ                  := nothing
--------------------------------------------------------------------------------------------
running Phase I...
      k |  lg(μ) |             φ |    |∇φ| |    |Δp| |       t |      tₗ | 

┌ Info: use exact Hessian from affine-constrained UMP
└ @ ExchangeMarket /Users/brent/workspace/ExchangeMarket.jl/src/algorithms/diff.jl:318
┌ Warning: linear constraint is not supported for affine scaling
└ @ ExchangeMarket /Users/brent/workspace/ExchangeMarket.jl/src/algorithms/hessianbar.jl:218


      2 |  -5.00 | +1.377065e+01 | 2.4e-02 | 5.9e-05 | 1.5e+01 | 1.4e+01 | 1.0e+00 | 7.1e-01 


┌ Info: use exact Hessian from affine-constrained UMP
└ @ ExchangeMarket /Users/brent/workspace/ExchangeMarket.jl/src/algorithms/diff.jl:318
┌ Warning: linear constraint is not supported for affine scaling
└ @ ExchangeMarket /Users/brent/workspace/ExchangeMarket.jl/src/algorithms/hessianbar.jl:218


      3 |  -5.00 | +1.377065e+01 | 3.1e-02 | 1.4e-06 | 2.2e+01 | 2.1e+01 | 1.0e+00 | 7.1e-01 


...

     28 |  -5.00 | +1.377065e+01 | 3.1e-02 | 2.5e-07 | 2.2e+02 | 2.1e+02 | 1.0e+00 | 7.1e-01 


┌ Info: use exact Hessian from affine-constrained UMP
└ @ ExchangeMarket /Users/brent/workspace/ExchangeMarket.jl/src/algorithms/diff.jl:318
┌ Warning: linear constraint is not supported for affine scaling
└ @ ExchangeMarket /Users/brent/workspace/ExchangeMarket.jl/src/algorithms/hessianbar.jl:218


     29 |  -5.00 | +1.377065e+01 | 3.1e-02 | 1.8e-07 | 2.2e+02 | 2.2e+02 | 1.0e+00 | 7.1e-01 
--------------------------------------------------------------------------------------------
 ✓  final play


┌ Info: use exact Hessian from affine-constrained UMP
└ @ ExchangeMarket /Users/brent/workspace/ExchangeMarket.jl/src/algorithms/diff.jl:318
┌ Warning: linear constraint is not supported for affine scaling
└ @ ExchangeMarket /Users/brent/workspace/ExchangeMarket.jl/src/algorithms/hessianbar.jl:218


 ✓  finished in          30 steps
             in 2.24456e+02 seconds
  best-resp. in 2.17864e+02 seconds 
            avg 7.26215e+00 seconds 
          usage 97.06%
--------------------------------------------------------------------------------------------
------------------------------------------------------------
 :problem size
 :    number of agents: 150
 :    number of goods: 210
 :    avg number of nonzero entries in c: 0.9514
 :equilibrium information
 :method: HessianBar
------------------------------------------------------------
10×2 DataFrame
 Row │ utility         left_budget  
     │ Float64         Float64      
─────┼──────────────────────────────
   1 │  80862.1         4.62148e-10
   2 │      1.06891e5   6.81713e-10
   3 │      3.00347e5   4.02235e-11
   4 │  30826.2         1.84157e-10
   5 │      4.39003e5   9.85523e-11
   6 │ 376075.0         2.12965e-10
   7 │      4.62274e5   5.36984e-10
   8 │   1861.11        1.27038e-10
   9 │      4.49553e5   1.40963e-10
  

In [37]:
f1.x

20×150 Matrix{Float64}:
 0.00893783    5.68985e-5   …  0.00160383  0.000918875  0.00084065
 0.000186242   1.64178e-8      0.00160383  0.000918875  0.00084065
 0.000191932   0.00213526      0.00160383  0.000918875  0.00084065
 4.77006e-6    3.10787e-6      0.00160383  0.000918875  0.00084065
 0.00494148    7.76815e-6      0.00160383  0.000918875  0.00084065
 0.0197525     0.00271779   …  0.00160383  0.000918875  0.00084065
 8.78681e-10   8.90989e-6      0.00160383  0.000918875  0.00084065
 0.00766202    0.00652043      0.00160383  0.000918875  0.00084065
 0.0111708     0.00051319      0.00160383  0.000918875  0.00084065
 0.00856893    0.00229138      0.00160383  0.000918875  0.00084065
 0.00044639    0.00039704   …  0.00160383  0.000918875  0.00084065
 2.38302e-6    0.000823166     0.00160383  0.000918875  0.00084065
 4.05288e-7    0.00107907      0.00160383  0.000918875  0.00084065
 0.000129946   0.000145046     0.00160383  0.000918875  0.00084065
 0.000249826   0.00130481      0.00160

## Test flow conservation

In [42]:
for i in 1:m
    _f = f1.x[:, i]
    _A = f1.constr_x[i].A
    @info "flow conservation residual:" _A * _f .|> abs |> sum
end

┌ Info: flow conservation residual:
│   (_A * _f .|> abs) |> sum = 2.400012697259535e-12
└ @ Main /Users/brent/Documents/recherche/25-ipm-auction-fisher/ExchangeMarket.jl/scripts/fisher/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y150sZmlsZQ==.jl:4
┌ Info: flow conservation residual:
│   (_A * _f .|> abs) |> sum = 1.580271265406996e-11
└ @ Main /Users/brent/Documents/recherche/25-ipm-auction-fisher/ExchangeMarket.jl/scripts/fisher/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y150sZmlsZQ==.jl:4
┌ Info: flow conservation residual:
│   (_A * _f .|> abs) |> sum = 5.701275411531246e-9
└ @ Main /Users/brent/Documents/recherche/25-ipm-auction-fisher/ExchangeMarket.jl/scripts/fisher/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y150sZmlsZQ==.jl:4
┌ Info: flow conservation residual:
│   (_A * _f .|> abs) |> sum = 4.786434322681998e-7
└ @ Main /Users/brent/Documents/recherche/25-ipm-auction-fisher/ExchangeMarket.jl/scripts/fisher/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y